# Building the design matrix

Importing librairies

In [1]:
import pandas as pd
import json
from glob import glob
import os.path as osp
from brainvisa import axon
axon.initializeProcesses()
import neuroHierarchy
from IPython.display import display_html
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rc('text', color='white')
matplotlib.rc('patch',facecolor='aliceblue', edgecolor='black', linewidth=1.0)



Loading toolbox baby
Loading toolbox brainrat
Loading toolbox connectomist
Loading toolbox cortical_surface
Loading toolbox data management
Loading toolbox datamind
Loading toolbox freesurfer
Loading toolbox morphologist
Loading toolbox nuclearimaging
Loading toolbox spm
Loading toolbox structural_analysis
Loading toolbox tms
Loading toolbox tools
Loading toolbox viewers
Loading toolbox My processes
The log file for this session is '/home/grg/.brainvisa/brainvisa7.log'
Database /home/grg/brainvisa/build/trunk/share/brainvisa-share-4.6 must be
updated because the database file is too old.
Database /home/grg/data/ALFA_FS_BV must be updated because the database file is
too old.
Database /home/grg/data/fs must be updated because the database file is too old.
ERROR No module named nuclearImaging.Interface_maker.GroupBoxCustom

--------------------------------------------------------------------------------
ImportError
processes.py (5042) in addDir:

toolbox=toolbox) # two arguments : proces

The imaging data and the metadata are located in the following folders.

The first contains the whole sequence of processed images, organized in an Axon database.

The second contains the images to give to SPM, as well as metadata, produced Excel sheets etc.

In [2]:
axondir = '/home/grg/data/ALFA_DWI'
spmdir = '/home/grg/spm/'

designmat_fp = osp.join(spmdir, 'designmatrix.xls')
designmat_fp2 = osp.join(spmdir, 'designmatrix2.xls')

The following function collects images that have been previously been uncompressed (see mdstats.copy_files), alongside quality score tables.

In [3]:
def collect_images(axondb, spmdir, verbose=True):
    images = {}
    for i, each in enumerate(['FA', 'MD', 'RD', 'L1']):
        items = glob(osp.join(spmdir, each, '*.nii'))
        items = dict([(osp.split(e)[-1][:5], e) for e in items])
        images[each] = items
    
    for i, each in enumerate(['qc_fa', 'qc_md', 'qc_l1', 'qc_rd']):
        types = ['Snapshots AAL on FA in MNI space Quality Scores',
                 'Snapshots AAL on MD in MNI space Quality Scores',
                 'Snapshots AAL on L1 in MNI space Quality Scores',
                 'Snapshots AAL on RD in MNI space Quality Scores']
        try:
            images[each] = pd.read_csv([e for e in axondb.findDiskItems(_type=types[i])][0].fullPath(), sep=';')
        except IndexError:
            images[each] = None
            if verbose:
                print types[i], 'returned no file'
    return images

... resulting in the following numbers.

In [4]:
axondb = neuroHierarchy.databases.database(axondir)
images = collect_images(axondb, spmdir=spmdir)

#subjects = list(set([e.get('subject') for e in axondb.findDiskItems(_type='ALFA T1 Raw Image') if not e.get('subject') is None]))
#subjects = dict([(e[:5], e) for e in subjects])

subjects = list(set([e.get('subject') for e in axondb.findDiskItems(_type='ALFA Mean Diffusivity Image') if not e.get('subject') is None]))
subjects = dict([(e[:5], e) for e in subjects])

# List of subjects removed based on neurological reports
bad = '10186, 10511, 10775, 12330, 12542, 12851, 44027, 55278, 66171, 66211, 10084, 11223, 55809, 10294, 10552, 13163, 13164, \
44027, 44086, 10586, 10947, 11016, 44733, 10448, 12156, 44086, 66294, 77051, 11526, 66029, 66215, 10966'.split(', ')

# Reading main table with metadata
demfp = osp.join(spmdir, 'data', 'Data_VBM_Alfa.csv')
# Contains age in days
agefp = osp.join(spmdir, 'data', 'age.json')
demo = pd.read_csv(demfp)
age = json.load(open(agefp))

# Reading ventricle volumes and TIV

tivs = dict([(str(int(e)), v) for e, v in demo[['Subj_ID', 'Tot Intracranial Volume (mm^3)']].to_dict(orient='split')['data']])

# FreeSurfer Volumes
vvol = pd.read_excel(osp.join(spmdir, 'data', 'aseg FS ALFA.xlsx'))
vvol_l = vvol[vvol['StructName'] == 'Left-Lateral-Ventricle']
vvol_r = vvol[vvol['StructName'] == 'Right-Lateral-Ventricle']
vvol_li = vvol[vvol['StructName'] == 'Left-Inf-Lat-Vent']
vvol_ri = vvol[vvol['StructName'] == 'Right-Inf-Lat-Vent']
vvol_3 = vvol[vvol['StructName'] == '3rd-Ventricle']
vvol_4 = vvol[vvol['StructName'] == '4th-Ventricle']

vvol_l = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_l[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])
vvol_r = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_r[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])
vvol_li = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_li[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])
vvol_ri = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_ri[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])
vvol_3 = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_3[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])
vvol_4 = dict([(str(int(e)), v / tivs[str(int(e))]) for e, v in vvol_4[['subject', 'Volume_mm3']].to_dict(orient='split')['data'] if str(int(e)) in tivs])

# Juando's Volumes
jdg = pd.read_excel(osp.join(spmdir, 'data', 'Ventricular volumes.xlsx'))
jdg_tivs = dict([(str(e)[:5], v*1000.0) for e, v in jdg[['subject', 'Total Intracranial Volume']].to_dict(orient='split')['data']])
vvol = dict([(str(e)[:5], v*1000.0 / tivs[str(e)[:5]]) for e, v in jdg[['subject', 'Ventricular Volume']].to_dict(orient='split')['data'] if str(e)[:5] in jdg_tivs])

In [5]:
columns = ['MD', 'FA', 'L1', 'RD', 'qc_md', 'qc_fa', 'qc_l1', 'qc_rd']
display_html(pd.DataFrame([len(images[k]) for k in columns], index=columns).transpose())

print '# of subjects found in Axon database:', len(subjects)
# List of subjects from the metadata table
rafsubj = [str(int(d['Subj_ID']))[:5] for i,d in demo.iterrows()]
print '# of subjects in Raf\'s table:', len(rafsubj)
print len(bad), 'subjects reported bad for neurological reasons'
for each in ['MD', 'FA', 'L1', 'RD']:
    qccode = 'qc_%s'%each.lower()
    bad_maps = [e for e in images[qccode]['Sujet'].tolist() 
                if images[qccode][images[qccode]['Sujet'] == e]['Note globale'].tolist()[0] < 3]
    print len(bad_maps), 'subjects with a bad %s map (score < 3)'%each, bad_maps

,MD,FA,L1,RD,qc_md,qc_fa,qc_l1,qc_rd
0,564,570,570,570,572,570,570,570


# of subjects found in Axon database: 570
# of subjects in Raf's table: 533
32 subjects reported bad for neurological reasons
7 subjects with a bad MD map (score < 3) ['11474', '11703', '13214', '66017', '10019', '10779', '55855']
3 subjects with a bad FA map (score < 3) ['10034', '10035', '55855']
4 subjects with a bad L1 map (score < 3) ['10486', '12914', '55855', '66017']
3 subjects with a bad RD map (score < 3) ['11474', '55855', '66017']


In [6]:
# Full list of headers
headers = list(demo.columns)
headers.insert(3, 'age_in_days')
headers.insert(1, 'image_id')
headers.extend(['FA', 'MD', 'L1', 'RD',
                  'age23', 'age24', 'age33', 'age34', 'age44',
                  'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44',
                  'ventricles_FS', 'ventricles_JDG',
                  'qc_fa', 'qc_md', 'qc_l1', 'qc_rd', 'is_bad'])

display_html(pd.DataFrame(headers))

,0
0,Subj_ID
1,image_id
2,Gender(0=female)
3,Age
4,age_in_days
5,Tot Intracranial Volume (mm^3)
6,Years of Education
7,Apoe2-2
8,Apoe2-3
9,Apoe2-4


Starting from the subjects from the metadata table, we add their corresponding images previously identified. Quality scores are also attached.

In [7]:
j = json.loads(demo.to_json(orient='index'))

# Complementing the table
no_image_found = []
for k,v in j.items():
    subjid = str(v['Subj_ID'])
    if not subjid[:5] in vvol:
        print subjid[:5], 'not found in ventricles'
        continue
    
    # Adding the images
    for t in ['FA', 'MD', 'L1', 'RD']:
        if not subjid in images[t].keys() or len(images[t][subjid]) == 0:
            no_image_found.append((subjid, t)) # 'no image found'
            v[t] = None
            continue    
        v[t] = images[t][subjid]
                                          
    # Adding the quality scores
    for t in ['qc_fa', 'qc_md', 'qc_l1', 'qc_rd']:
        if images[t] is None:
            v[t] = None
            continue
        v[t] = images[t][images[t]['Sujet'] == subjects[subjid]]['Note globale'].tolist()[0] \
                         if subjid in subjects.keys() and subjects[subjid] in images[t]['Sujet'].tolist() else None 
         
    v['age_in_days'] = age[subjid] / 365.25
    v['image_id'] = subjects.get(subjid, None)
    v['is_bad'] = subjid in bad
    v['ventricles_FS'] = vvol_l[subjid] + vvol_r[subjid] + vvol_li[subjid] + vvol_ri[subjid]# + vvol_3[subjid] + vvol_4[subjid]
    v['ventricles_JDG'] = vvol[subjid[:5]]
    print subjid, v['ventricles_FS'], v['ventricles_JDG']
    

    
    for each in ['Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4']:
        f = each.replace('Apoe', 'age').replace('-','')
        v[f] = v['age_in_days'] * int(v[each])
        f = each.replace('Apoe', 'agesq').replace('-','')
        v[f] = v['age_in_days']**2 * int(v[each])

# Storing all this in a pandas DataFrame
data = pd.read_json(json.dumps(j), orient='index')
data = pd.DataFrame(data.sort_values(by='Subj_ID'), columns=headers, index=xrange(len(data.index))).dropna(0)
data = data.sort_values(['Apoe2-2', 'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 'Subj_ID'], 
                        ascending=[False, False, False, False, False, False, True])

data.to_excel(designmat_fp)

diffsub = sorted(set(rafsubj).difference(set([str(e) for e in data['Subj_ID']])))

print 'Subjects with no image found:', no_image_found
print [e[0] for e in no_image_found], '\n'
print 'Subjects from the initial table that were not included in the final one:', diffsub

11219 12.893767088 12.3404722695
10010 not found in ventricles
11245 12.8613388662 12.4842595287
11247 5.83054444475 5.23740364831
11191 7.38255383255 6.6080707323
11195 6.94174805121 6.90196920965
11196 8.0706344182 7.31643853918
11201 6.40466880734 5.89779985985
11252 5.25931791692 5.11793243361
11254 9.13369547619 8.78562563231
10482 10.7120150522 10.3688577172
10486 20.8334586489 19.5553084121
10453 16.4575186386 15.0655843855
10461 16.9047255757 16.1181205714
10436 16.322983872 15.5733925871
10450 28.8261434997 25.9880961943
10418 12.8575593865 12.011912891
10433 11.1259926928 10.1729550215
10397 8.5704226631 8.38958718491
10416 21.6879181629 20.0063298023
44068 22.0368052433 20.7521508701
44046 11.818886039 10.4779379202
21057 14.5921848295 435.299498591
21042 5.63752830272 4.90122183302
21092 14.964386094 14.3393850312
21073 14.8284514206 13.5507172218
13312 17.8342789677 17.6958738621
13242 6.8385361486 5.98007910405
21026 16.8164222312 15.8218194544
13367 6.87149700703 6.52757

Finally the result is:

In [8]:
# Removing the 7 first 2-2 homozygotes
data = data.drop(data.index[0:7])
headers2 = ['Subj_ID', 'MD', 'FA', 'L1', 'RD',
            'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 'age23', 'age24', 'age33', 'age34', 
            'age44', 'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 'ventricles_FS', 'ventricles_JDG', 'Years of Education', 'Gender(0=female)', 
            'qc_fa', 'qc_md', 'qc_l1', 'qc_rd', 'is_bad']
data = pd.DataFrame(data, columns=headers2)
data.to_excel(designmat_fp2)
data

,Subj_ID,MD,FA,L1,RD,Apoe2-3,Apoe2-4,Apoe3-3,Apoe3-4,Apoe4-4,...,agesq44,ventricles_FS,ventricles_JDG,Years of Education,Gender(0=female),qc_fa,qc_md,qc_l1,qc_rd,is_bad
15,10070,/home/grg/spm/MD/10070_MD_MNIspace_s.nii,/home/grg/spm/FA/10070_FA_MNIspace_s.nii,/home/grg/spm/L1/10070_L1_MNIspace_s.nii,/home/grg/spm/RD/10070_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,4.520876,4.066362,10,0,4.0,4.0,4.0,4.0,0.0
17,10102,/home/grg/spm/MD/10102_MD_MNIspace_s.nii,/home/grg/spm/FA/10102_FA_MNIspace_s.nii,/home/grg/spm/L1/10102_L1_MNIspace_s.nii,/home/grg/spm/RD/10102_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,16.507219,15.301898,18,1,4.0,4.0,4.0,4.0,0.0
18,10108,/home/grg/spm/MD/10108_MD_MNIspace_s.nii,/home/grg/spm/FA/10108_FA_MNIspace_s.nii,/home/grg/spm/L1/10108_L1_MNIspace_s.nii,/home/grg/spm/RD/10108_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,10.421616,9.432586,15,0,4.0,4.0,4.0,4.0,0.0
33,10235,/home/grg/spm/MD/10235_MD_MNIspace_s.nii,/home/grg/spm/FA/10235_FA_MNIspace_s.nii,/home/grg/spm/L1/10235_L1_MNIspace_s.nii,/home/grg/spm/RD/10235_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,7.564201,7.059961,8,0,4.0,4.0,4.0,4.0,0.0
51,10365,/home/grg/spm/MD/10365_MD_MNIspace_s.nii,/home/grg/spm/FA/10365_FA_MNIspace_s.nii,/home/grg/spm/L1/10365_L1_MNIspace_s.nii,/home/grg/spm/RD/10365_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,7.770477,7.754194,12,0,4.0,4.0,4.0,4.0,0.0
56,10419,/home/grg/spm/MD/10419_MD_MNIspace_s.nii,/home/grg/spm/FA/10419_FA_MNIspace_s.nii,/home/grg/spm/L1/10419_L1_MNIspace_s.nii,/home/grg/spm/RD/10419_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,3.484049,3.298463,8,0,4.0,4.0,4.0,4.0,0.0
58,10463,/home/grg/spm/MD/10463_MD_MNIspace_s.nii,/home/grg/spm/FA/10463_FA_MNIspace_s.nii,/home/grg/spm/L1/10463_L1_MNIspace_s.nii,/home/grg/spm/RD/10463_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,7.401804,6.594490,18,0,4.0,4.0,4.0,4.0,0.0
60,10530,/home/grg/spm/MD/10530_MD_MNIspace_s.nii,/home/grg/spm/FA/10530_FA_MNIspace_s.nii,/home/grg/spm/L1/10530_L1_MNIspace_s.nii,/home/grg/spm/RD/10530_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,7.521816,7.626067,15,0,4.0,4.0,4.0,4.0,0.0
61,10551,/home/grg/spm/MD/10551_MD_MNIspace_s.nii,/home/grg/spm/FA/10551_FA_MNIspace_s.nii,/home/grg/spm/L1/10551_L1_MNIspace_s.nii,/home/grg/spm/RD/10551_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,17.226243,15.809334,12,1,4.0,4.0,4.0,4.0,0.0
62,10563,/home/grg/spm/MD/10563_MD_MNIspace_s.nii,/home/grg/spm/FA/10563_FA_MNIspace_s.nii,/home/grg/spm/L1/10563_L1_MNIspace_s.nii,/home/grg/spm/RD/10563_RD_MNIspace_s.nii,1,0,0,0,0,...,0.000000,6.444551,6.276600,12,0,4.0,4.0,4.0,4.0,0.0


Analysis ready to be run.

In [9]:
import sys
sys.path.append('/home/grg/git/alfa/')

In [10]:
import multireg_spm12 as mreg

excel_fp = '/home/grg/spm/data/designmatrix2.xls' # table containing model data
paramtype = 'MD' # given column from the previous table
analysis_wd = '/tmp/dummy' # working directory
mask_fp = '/home/grg/spm/MNI_T1_brain_mask.nii' # explicit mask for the analysis
print 'Reading', excel_fp, 'looking up column', paramtype, '...'

a = mreg.multiple_regression_analysis(paramtype, excel_fp, analysis_wd, mask_fp, verbose=False)
#a.run()

Reading /home/grg/spm/data/designmatrix2.xls looking up column MD ...
